In [1]:
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import time
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor
from torchvision import datasets, models, transforms
from torchsummary import summary
import matplotlib.pyplot as plt
import numpy as np
from torchvision.models import AlexNet_Weights
import os

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") # device object
print(device)

cuda:0


In [2]:
transforms_train = transforms.Compose([
    transforms.Resize((224, 224)),   #must same as here
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(), # data augmentation
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) # normalization
])
transforms_test = transforms.Compose([
    transforms.Resize((224, 224)),   #must same as here
     transforms.CenterCrop((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [3]:
data_dir = "dataset/caltech256/256_ObjectCategories" 
dataset = datasets.ImageFolder(data_dir, transforms_train)

data_dir = "Min_number_stop" 
stop = datasets.ImageFolder(data_dir, transforms_train)

In [4]:
class_names = dataset.classes
class_num = len(class_names)
print("Number of classes Caltech256: ", class_num)
stop_class = stop.classes
class_num = len(stop_class)
print("Number of class stop directory: ", class_num)
print("Number of images in dataset: ", len(dataset))

Number of classes Caltech256:  257
Number of class stop directory:  1
Number of images in dataset:  30614


In [5]:
train_size = int(0.25 * len(dataset))
test_size = len(dataset) - train_size
train1, _ = torch.utils.data.random_split(dataset, [train_size, test_size])
print('Train (1st subset) size:', len(train1))

Train (1st subset) size: 7653


In [6]:
train_size = int(0.5 * len(dataset))
test_size = len(dataset) - train_size
train2, _ = torch.utils.data.random_split(dataset, [train_size, test_size])
print('Train (2st subset) size:', len(train2))

Train (2st subset) size: 15307


In [7]:
train_size = int(0.75 * len(dataset))
test_size = len(dataset) - train_size
train3, _ = torch.utils.data.random_split(dataset, [train_size, test_size])
print('Train (2st subset) size:', len(train3))

Train (2st subset) size: 22960


In [8]:
train_size = int(0.9 * len(dataset))
test_size = len(dataset) - train_size
train4, _ = torch.utils.data.random_split(dataset, [train_size, test_size])
print('Train (2st subset) size:', len(train4))

Train (2st subset) size: 27552


In [9]:
train1_dataloader =  torch.utils.data.DataLoader(train1, batch_size=12, shuffle=True, num_workers=8)
train2_dataloader =  torch.utils.data.DataLoader(train2, batch_size=12, shuffle=True, num_workers=8)
train3_dataloader =  torch.utils.data.DataLoader(train3, batch_size=12, shuffle=True, num_workers=8)
train4_dataloader =  torch.utils.data.DataLoader(train4, batch_size=12, shuffle=True, num_workers=8)

In [10]:
train_size = int(0.25 * len(stop))
test_size = len(stop) - train_size
train_stop_1, test_stop_1 = torch.utils.data.random_split(stop, [train_size, test_size])
print('Stop train dataset size:', len(train_stop_1))
print('Stop test dataset size:', len(test_stop_1))

Stop train dataset size: 16
Stop test dataset size: 48


In [11]:
train_size = int(0.5 * len(stop))
test_size = len(stop) - train_size
train_stop_2, test_stop_2 = torch.utils.data.random_split(stop, [train_size, test_size])
print('Stop train dataset size:', len(train_stop_2))
print('Stop test dataset size:', len(test_stop_2))

Stop train dataset size: 32
Stop test dataset size: 32


In [12]:
train_size = int(0.75 * len(stop))
test_size = len(stop) - train_size
train_stop_3, test_stop_3 = torch.utils.data.random_split(stop, [train_size, test_size])
print('Stop train dataset size:', len(train_stop_3))
print('Stop test dataset size:', len(test_stop_3))

Stop train dataset size: 48
Stop test dataset size: 16


In [13]:
train_size = int(0.9 * len(stop))
test_size = len(stop) - train_size
train_stop_4, test_stop_4 = torch.utils.data.random_split(stop, [train_size, test_size])
print('Stop train dataset size:', len(train_stop_4))
print('Stop test dataset size:', len(test_stop_4))

Stop train dataset size: 57
Stop test dataset size: 7


In [14]:
train1_stop =  torch.utils.data.DataLoader(train_stop_1, batch_size=12, shuffle=True, num_workers=8)
test1_stop = torch.utils.data.DataLoader(test_stop_1, batch_size=12)
train2_stop =  torch.utils.data.DataLoader(train_stop_2, batch_size=12, shuffle=True, num_workers=8)
test2_stop = torch.utils.data.DataLoader(test_stop_2, batch_size=12)
train3_stop =  torch.utils.data.DataLoader(train_stop_3, batch_size=12, shuffle=True, num_workers=8)
test3_stop = torch.utils.data.DataLoader(test_stop_3, batch_size=12)
train4_stop =  torch.utils.data.DataLoader(train_stop_4, batch_size=12, shuffle=True, num_workers=8)
test4_stop = torch.utils.data.DataLoader(test_stop_4, batch_size=12)

In [15]:
model = models.alexnet(weights = AlexNet_Weights.IMAGENET1K_V1)  
# Freeze all the pre-trained layers
for param in model.parameters():
    param.requires_grad = False
    
model.classifier = nn.Sequential(*[model.classifier[i] for i in range(5)])
model.to(device)

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [16]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

knn = KNeighborsClassifier(n_neighbors=12)

In [22]:
accuracy_train = []
features = []
f_labels = []

for i, data in enumerate(train1_dataloader):
    image, labels = data
    image = image.to(device)
    labels = labels.to(device)

    batch_features = model(image)

    for j in range(image.size()[0]):
        features.append(batch_features[j].cpu().numpy())
        f_labels.append(labels[j].cpu().numpy()) 

print(f"Number of feature vectors for train set (before tag insertion): {len(features)}")

for i, data in enumerate(train1_stop):
    image, labels = data
    image = image.to(device)
    labels = labels.to(device)

    batch_features = model(image)

    for j in range(image.size()[0]):
        features.append(batch_features[j].cpu().numpy())
        f_labels.append(labels[j].cpu().numpy())

print(f"Number of feature vectors for train set (after tag insertion): {len(features)}")
knn.fit(features, f_labels)

for i in range(100):
    test_features = []
    test_labels = []

    for i, data in enumerate(test1_stop):
        image, labels = data
        image = image.to(device)
        labels = labels.to(device)

        batch_features = model(image)

        for j in range(image.size()[0]):
            test_features.append(batch_features[j].cpu().numpy())
            test_labels.append(labels[j].cpu().numpy())

    pred = knn.predict(test_features)
    accuracy = accuracy_score(test_labels, pred)
    accuracy_train.append(accuracy)
    #print("Knn test accuracy:", accuracy)

print("Knn average test accuracy:", np.mean(accuracy_train))

Number of feature vectors for train set (before tag insertion): 7653
Number of feature vectors for train set (after tag insertion): 7669
Knn average test accuracy: 0.6225
